# Summary

In [ ]:
#!pip -q install missingno optuna pendulum loguru ydata_profiling yellowbrick mlflow xgboost tensorflow pycaret boto3

# Librairies

In [ ]:
# reload modules before executing user code.
%reload_ext autoreload
%autoreload 2

import boto3
import os
import sys
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pendulum
import pandas as pd
import plotly.express as px
import seaborn as sns
from pathlib import Path


from loguru import logger

sys.path.append(str(Path.cwd().parent))
from settings.params import (DATA_DIR_INPUT,
                             DATA_DIR_OUTPUT,
                             MODEL_PARAMS,
                             REPORT_DIR,
                             RUNS_DIR,
                             TIMEZONE,
                             MODEL_DIR
                            )
from src.make_dataset import load_data

pd.set_option("display.max_columns", None)

# Settings

In [ ]:
# time in UTC
log_fmt = ("<green>{time:YYYY-MM-DD HH:mm:ss.SSS!UTC}</green> | <level>{level: <8}</level> | "
           "<cyan>{name}</cyan>:<cyan>{function}</cyan>:<cyan>{line}</cyan> - {message}"
          )
log_config = {
    "handlers": [
        {"sink": sys.stderr, "format": log_fmt},
    ],
}
logger.configure(**log_config)

In [ ]:
EXECUTION_DATE = pendulum.now(tz=TIMEZONE)

logger.info(f"Execution date: {EXECUTION_DATE}")

logger.info(f"\nData input directory : {DATA_DIR_INPUT}\nData output directory: {DATA_DIR_OUTPUT}")


# model parameters
FEATURES = MODEL_PARAMS["FEATURES"]
TARGET_NAME = MODEL_PARAMS["TARGET"]